In [13]:
import pandas as pd
import numpy as np

def create_main_fields(care_areas_df, main_field_size):
    """Generate main fields centered around each care area using the diagonal approach."""
    main_fields = []
    main_field_id = 0

    for _, care_area in care_areas_df.iterrows():
        center_x = (care_area["Xmin"] + care_area["Xmax"]) / 2
        center_y = (care_area["Ymin"] + care_area["Ymax"]) / 2

        main_fields.append({
            "ID": main_field_id,
            "Xmin": center_x - main_field_size / 2,
            "Xmax": center_x + main_field_size / 2,
            "Ymin": center_y - main_field_size / 2,
            "Ymax": center_y + main_field_size / 2
        })
        main_field_id += 1

    return pd.DataFrame(main_fields)

def calculate_overlap(care_area, main_field):
    """Calculate the overlapping area between a care area and a main field."""
    x_min_overlap = max(care_area["Xmin"], main_field["Xmin"])
    x_max_overlap = min(care_area["Xmax"], main_field["Xmax"])
    y_min_overlap = max(care_area["Ymin"], main_field["Ymin"])
    y_max_overlap = min(care_area["Ymax"], main_field["Ymax"])
    
    if x_min_overlap < x_max_overlap and y_min_overlap < y_max_overlap:
        return {"Xmin": x_min_overlap, "Xmax": x_max_overlap, "Ymin": y_min_overlap, "Ymax": y_max_overlap}
    return None

def create_subfields(main_fields_df, care_areas_df, sub_field_size1, sub_field_size2):
    """Generate subfields within the overlapping regions of main fields and care areas."""
    sub_fields = []

    for _, main_field in main_fields_df.iterrows():
        for _, care_area in care_areas_df.iterrows():
            overlap_rect = calculate_overlap(care_area, main_field)
            if overlap_rect:
                sub_x_min, sub_y_min = overlap_rect["Xmin"], overlap_rect["Ymin"]
                sub_x_max, sub_y_max = overlap_rect["Xmax"], overlap_rect["Ymax"]

                # Cover with the largest subfield size first
                num_x_subfields_large = int(np.ceil((sub_x_max - sub_x_min) / sub_field_size1))
                num_y_subfields_large = int(np.ceil((sub_y_max - sub_y_min) / sub_field_size1))

                for i in range(num_x_subfields_large):
                    for j in range(num_y_subfields_large):
                        sub_x = sub_x_min + i * sub_field_size1
                        sub_y = sub_y_min + j * sub_field_size1

                        # Check boundaries
                        if (sub_x + sub_field_size1 <= sub_x_max) and (sub_y + sub_field_size1 <= sub_y_max):
                            sub_fields.append({
                                "ID": len(sub_fields),
                                "Xmin": sub_x,
                                "Xmax": sub_x + sub_field_size1,
                                "Ymin": sub_y,
                                "Ymax": sub_y + sub_field_size1,
                                "Main Field ID": int(main_field['ID']),
                                "Sub Field Size": sub_field_size1
                            })

                # Check for remaining area that is not covered by the large subfields
                remaining_x_max = sub_x_max
                remaining_y_max = sub_y_max

                if sub_x + sub_field_size1 < remaining_x_max:
                    num_x_subfields_small = int(np.ceil((remaining_x_max - sub_x) / sub_field_size2))
                    for i in range(num_x_subfields_small):
                        sub_x = sub_x + i * sub_field_size2
                        if sub_x + sub_field_size2 <= remaining_x_max:
                            sub_fields.append({
                                "ID": len(sub_fields),
                                "Xmin": sub_x,
                                "Xmax": sub_x + sub_field_size2,
                                "Ymin": sub_y_min,
                                "Ymax": sub_y_min + sub_field_size1,
                                "Main Field ID": int(main_field['ID']),
                                "Sub Field Size": sub_field_size2
                            })

                if sub_y + sub_field_size1 < remaining_y_max:
                    num_y_subfields_small = int(np.ceil((remaining_y_max - sub_y) / sub_field_size2))
                    for j in range(num_y_subfields_small):
                        sub_y = sub_y + j * sub_field_size2
                        if sub_y + sub_field_size2 <= remaining_y_max:
                            sub_fields.append({
                                "ID": len(sub_fields),
                                "Xmin": sub_x_min,
                                "Xmax": sub_x_min + sub_field_size1,
                                "Ymin": sub_y,
                                "Ymax": sub_y + sub_field_size2,
                                "Main Field ID": int(main_field['ID']),
                                "Sub Field Size": sub_field_size2
                            })

    return pd.DataFrame(sub_fields)

def main():
    try:
        # Load the care areas CSV file
        care_areas_df = pd.read_csv(
            r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\3rd\CareAreas.csv",
            names=["ID", "Xmin", "Xmax", "Ymin", "Ymax"],
            skiprows=1
        )

        # Load and clean the metadata CSV file
        metadata_df = pd.read_csv(
            r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\3rd\metadata.csv",
            header=None,
            skip_blank_lines=True
        )
        metadata_df = metadata_df.apply(pd.to_numeric, errors='coerce').dropna()

        if len(metadata_df) < 2:
            raise ValueError("Metadata file must contain at least two rows for Main Field and Sub Field sizes.")

        main_field_size = metadata_df.iloc[0, 0]
        sub_field_size1 = metadata_df.iloc[1, 0]  # Maximum size
        sub_field_size2 = metadata_df.iloc[1, 1]  # Minimum size

        # Create main fields and subfields
        main_fields_df = create_main_fields(care_areas_df, main_field_size)
        sub_fields_df = create_subfields(main_fields_df, care_areas_df, sub_field_size1, sub_field_size2)

        # Print DataFrames
        print("Main Fields DataFrame:")
        print(main_fields_df)

        print("\nSub Fields DataFrame:")
        print(sub_fields_df)

        # Save DataFrames to CSV files with header=False
        main_fields_df.to_csv(r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\MainFields.csv", index=False, header=False)
        sub_fields_df.to_csv(r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\SubFields.csv", index=False, header=False)

        print("\nMain fields and subfields saved successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Main Fields DataFrame:
      ID    Xmin    Xmax    Ymin    Ymax
0      0    -5.5    94.5   174.5   274.5
1      1    -5.5    94.5   354.5   454.5
2      2    -5.5    94.5   534.5   634.5
3      3    -5.5    94.5   714.5   814.5
4      4    -5.5    94.5   894.5   994.5
..   ...     ...     ...     ...     ...
523  523  3954.5  4054.5  3234.5  3334.5
524  524  3954.5  4054.5  3414.5  3514.5
525  525  3954.5  4054.5  3594.5  3694.5
526  526  3954.5  4054.5  3774.5  3874.5
527  527  3954.5  4054.5  3954.5  4054.5

[528 rows x 5 columns]

Sub Fields DataFrame:
Empty DataFrame
Columns: []
Index: []

Main fields and subfields saved successfully.
